# Load and clean data

In [645]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import datetime

In [646]:
path = "X" # sensitive information

with open(path, 'r') as j:
     articles = json.loads(j.read())
        
raw = pd.DataFrame(articles)


In [647]:
# Single out "%Y-%m-%d" in published 
raw['published'] = raw['published'].map(lambda t:datetime.datetime.strptime(t,"%Y-%m-%dT%H:%M:%SZ"))

# Split section and subsection into two seperate columns
# if no subsection is available, replace with None  
sections = raw['section'].items()
section = []
subsection = []
for i,v in sections:
    if len(v.keys()) > 1:
        section.append(list(v.values())[0])
        subsection.append(list(v.values())[1])
    else:
        section.append(list(v.values())[0])
        subsection.append(None)
        
raw['section'] = section
raw['subsection'] = subsection

In [648]:
from collections import Counter
body0 = Counter(raw[raw['body'].map(lambda x: len(x.split())) == 0]['section'])
body0 = dict(sorted(body0.items(), key=lambda item: -item[1]))

# Create cleaned df
raw = raw[raw['published'] < '2021-11-01']
raw = raw[raw['section'] != 'X'] # sensitive information
raw = raw[raw['section'] != 'X'] # sensitive information
raw = raw[raw['section'] != 'X'] # sensitive information
raw = raw[raw['section'] != 'X'] # sensitive information
# remove extrem values 
raw = raw[raw['body'].map(lambda x: len(x.split())) > 50]
df = raw[raw['body'].map(lambda x: len(x.split())) < 1500]


# Descriptive statistics

In [649]:
# The remaining clean data have the following statistics:
#print(f'The remaining number of observations are: {len(df)} \n')
section_count = Counter(df['section'])
section_count = dict(sorted(section_count.items(), key=lambda item: -item[1]))
#print(f'The remaining number of sections are: {len(section_count)} \n')
#print('Section remaining in cleaned data:')
#display(section_count)


subsection_count = Counter(df['subsection'])
subsection_count = dict(sorted(subsection_count.items(), key=lambda item: -item[1]))
#print(f'The remaining number of subsection are: {len(subsection_count)-1} \n')
#print('Subsection remaining in cleaned data:')
#display(subsection_count)

In [ ]:
plt.bar(section_count.keys(), section_count.values())
plt.xticks(rotation=-90)
plt.title("Number of published articles per section")
plt.show()

In [ ]:
pd.DataFrame([subsection_count]).T.plot.bar(legend=False)
plt.xticks(rotation=-90)
plt.title("Number of published articles per subsection")
plt.show()

Based on the counts in each section there seems to be a large difference in terms publications, lets therefore create two new section categories: *section_large* and *section_small* which is decided for a threshold $P$ so that clearer inference of the clusters can be drawn. This implies mergeing some of the subsections with the sections.

In [652]:
p = 100
section_large = []
section_small = []
for row in df.itertuples():
    if(section_count.get(row.section) >= p):
        if(subsection_count.get(row.subsection) >= p and row.subsection != None):
            section_large.append(row.subsection)
            section_small.append(None)
        else:
            section_large.append(row.section)
            section_small.append(None)
    else:
        if(row.subsection != None):
            section_small.append(row.subsection)
            section_large.append(None)
        else:
            section_small.append(row.section)
            section_large.append(None)

df.insert(2, "section_large", section_large, True)
df.insert(2, "section_small", section_small, True)

In [653]:
section_large_count = Counter(df['section_large'])
section_large_count = dict(sorted(section_large_count.items(), key=lambda item: -item[1]))
#print(f'The remaining number of sections are: {len(section_large_count)} \n')
#print('Section remaining in cleaned data:')
#display(section_large_count)

In [ ]:

pd.DataFrame([section_large_count]).T.plot.bar(legend=False)
plt.xticks(rotation=-90)
plt.title("Number of published articles per aggregated section")
plt.show()

In [ ]:
monthly_pub = Counter(df['published'].map(lambda x: x.strftime("%Y-%m")))

plt.bar(monthly_pub.keys(), monthly_pub.values())
plt.xticks(rotation=-90)
plt.title("Number of published articles per month")
plt.show()

In [13]:
# descriptive statistics with respect to the words in the body
#print(df['body'].map(lambda x: len(x.split())).describe())
# descriptive statistics with respect to the words in the body
# group on section
#display(df['body'].map(lambda x: len(x.split())).groupby(df['section_large']).describe())

# Preprocessing

## Split data

In [14]:
# 
from nltk import word_tokenize
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

# Preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

# Model 
from hdbscan import HDBSCAN
from hdbscan import approximate_predict

# Validation
from hdbscan.validity import validity_index

# Purity
from sklearn import metrics
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)


# To save and calculate the calculation time
import time

def preprocess(text):
    # lowercase
    text = text.lower()
    # remove html tags
    text_p = re.sub('<[^<]+?>', '',text)
    # remove special symbols
    text_p = "".join([char for char in text_p if char not in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~–”…▪“•✓►’'])
    # tokenize
    words = word_tokenize(text_p,language="swedish")
    # remove swedish stopwords
    stop_words = stopwords.words('swedish')
    filtered_words = [word for word in words if word not in stop_words]
    # removing tokens that only contains numbers and alphabetic 
    filtered_words = [w for w in filtered_words if w.isalnum()] 
        
    return filtered_words


In [ ]:
# create the weeks 
week = df['published'].map(lambda x: x.isocalendar()[1])
week = np.where(week == 53, 0, week)
weeks = np.unique(week)
week_pub = Counter(week)
plt.figure(figsize=(15,7))
plt.bar(week_pub.keys(), week_pub.values())
plt.xticks(rotation=-90)
plt.xticks(weeks)
plt.title("Number of published articles per week")
plt.show()

In [ ]:
window_size = 8
# dataframe to save each week score into
d0 = pd.DataFrame(columns = ['n_components','min_cluster_size','DBCV_res','n_noise','n_clusters'])
# loop over every week
for w in weeks[:-window_size+1]:

    # Select the window for prior information
    train = df[(week >= w) & (week < (w + window_size))].copy()
    
    start = time.time()
    vectorizer = TfidfVectorizer(tokenizer=preprocess,
                                 max_features=int(1e5)
                                ).fit(train['body'])
    
    tfidf = vectorizer.transform(train['body'])
    end = time.time()
    print(end - start)

    n_components_list = [75]
    min_cluster_size_list = [min_cluster_size for min_cluster_size in range(5,31)]

    n_components_res = []
    min_cluster_size_res = []
    DBCV_res = []
    silhouette_res = []
    n_noise_res = []
    n_clusters_res = []

    start = time.time()
    # Iterate over n_components (LSA)
    for n_comp in n_components_list:
        print(f'Iteration n_comp: {n_comp}')
        # Apply latent semantic analysis (LSA)
        LSA = TruncatedSVD(n_comp,n_iter=20,random_state=123).fit(tfidf)
        # Normalise the LSA embeddings to Angular distance
        X_train = normalize(LSA.transform(tfidf),norm='l2')

        # Iterate over minimum cluster size (hdbscan)
        for min_clust in min_cluster_size_list:
            print(f'Iteration min_clust: {min_clust}')
            # Train the hdbscan model
            hdbscan = HDBSCAN(min_clust,prediction_data=False).fit(X_train)
            # Evaluate hdbscan with Density-Based Clustering Validation (DBCV)
            DBCV_res.append(validity_index(X_train,hdbscan.labels_))
            # Count noise points in each setup
            n_noise_res.append(sum(hdbscan.labels_==-1))
            # Count number of clusters
            n_clusters_res.append(len(np.unique(hdbscan.labels_)))
            
            # Setup used
            n_components_res.append(n_comp)
            min_cluster_size_res.append(min_clust)
            
    end = time.time()
    print(end - start)

    d1 = pd.DataFrame({'n_components':n_components_res,
                                'min_cluster_size':min_cluster_size_res,
                                'DBCV_res':DBCV_res,
                                'n_noise':n_noise_res,
                                'n_clusters':n_clusters_res})
    # Save evaluation to 
    d0 = pd.concat([d0,d1])
#   


## Evaluation

In [ ]:
opt_min_cluster_size = 10
avg_res = d0.groupby('min_cluster_size').mean()
plt.figure(figsize=(10,7))
plt.scatter(avg_res.index,avg_res['DBCV_res'])
plt.axvline(x=opt_min_cluster_size,c='red')
plt.xlabel("MinPts")
plt.ylabel("Avg. DBCV")
plt.title("Average DBCV over MinPts")
plt.show()

## Investigate clusters

In [ ]:
# setup
freq = []
n_clusters = []

section_names = list(df.section_large.unique())
section_names = ['None' if v is None else v for v in section_names]

obs_count_w = pd.DataFrame(columns=section_names)
cluster_count_w = pd.DataFrame(columns=section_names)

window_size=8
for w in weeks[:-window_size+1]:
    print(w+window_size-1)
    # Select the window for prior information
    train = df[(week >= w) & (week < (w + window_size))].copy()
    train_section_large = pd.DataFrame([str(s) for s in train['section_large']])

    start = time.time()
    vectorizer = TfidfVectorizer(tokenizer=preprocess,
                                 max_features=int(1e5)
                                ).fit(train['body'])
    n_comp = 75
    tfidf = vectorizer.transform(train['body'])
    LSA = TruncatedSVD(n_comp,n_iter=20,random_state=123).fit(tfidf)
    X_train = normalize(LSA.transform(tfidf),norm='l2')

    hdbscan = HDBSCAN(min_cluster_size=opt_min_cluster_size).fit(X_train)

    # number of observations and clusters found per week
    freq.append(sum(hdbscan.labels_ == -1)/len(hdbscan.labels_))
    n_clusters.append(len(np.unique(hdbscan.labels_))-1)

    # number of observation for each section
    obs_count = dict(Counter(train_section_large[0].values[hdbscan.labels_ !=-1]))

    # number of clusters in each section
    cluster_count = [(i,j) for i,j in zip(train_section_large[0].values[hdbscan.labels_ !=-1],
                          hdbscan.labels_[hdbscan.labels_ !=-1])]
    cluster_count = dict(Counter([i for i,j in Counter(cluster_count).keys()]))

    # Summary of the result
    obs_count_w = pd.concat([obs_count_w,pd.DataFrame(obs_count, index=[w+window_size-1])])

    cluster_count_w = pd.concat([cluster_count_w,pd.DataFrame(cluster_count, index=[w+window_size-1])])
    



In [ ]:
w = [w for w in range(7,44)]
plt.figure(figsize=(8,5))
plt.plot(w,freq)
plt.title("Percentage of clustered observation (Not noise)")
plt.xlabel("Window")
plt.ylabel("% clustered observations")
plt.show()


w = [w for w in range(7,44)]
plt.figure(figsize=(8,5))
plt.plot(w,n_clusters)
plt.title("Number of clusters")
plt.xlabel("Window")
plt.ylabel("Number of clusters")
plt.show()

In [ ]:
import seaborn as sns
# heatmap obs_count
obs_count_w.fillna(0, inplace=True)
cluster_count_w.fillna(0, inplace=True)
plt.figure(figsize=(10,7))
sns.heatmap(obs_count_w.T,
            cmap="viridis",
            annot_kws={'fontsize':11},
            square=True,
            linewidth=0.01,
            linecolor="#222")
plt.xlabel("Window")
plt.ylabel("Section")
plt.title("Number of articles clustered per section")
plt.show()

# heatmap cluster_count
plt.figure(figsize=(10,7))
sns.heatmap(cluster_count_w.T,
            cmap="viridis",
            annot_kws={'fontsize':11},
            square=True,
            linewidth=0.01,
            linecolor="#222")
plt.ylabel("Section")
plt.xlabel("Window")
plt.title("Number of clusters found per section")
plt.show()

In [ ]:
# evaluation of 3 time periods 
for w in [0,18,36]:
    # redo the clustering
    train = df[(week >= w) & (week < (w + window_size))].copy()
    train_section_large = pd.DataFrame([str(s) for s in train['section_large']])

    start = time.time()
    vectorizer = TfidfVectorizer(tokenizer=preprocess,
                                 max_features=int(1e5)
                                ).fit(train['body'])
    n_comp = 75
    tfidf = vectorizer.transform(train['body'])
    LSA = TruncatedSVD(n_comp,n_iter=20,random_state=123).fit(tfidf)
    X_train = normalize(LSA.transform(tfidf),norm='l2')

    hdbscan = HDBSCAN(min_cluster_size=opt_min_cluster_size).fit(X_train)


    section_names = ['None' if v is None else v for v in section_large]
    # construct data for heatmap
    cf = pd.DataFrame(index=np.unique(section_names), columns= np.unique(hdbscan.labels_))
    
    vals = dict(Counter([(i,j) for i,j in zip(train_section_large[0].values,hdbscan.labels_)]))

    for k in vals.keys():
        cf[k[1]].loc[[k[0]]] = vals.get(k)

    cf.fillna(0, inplace=True)
    # relative frequency 
    cf = cf/cf.sum(axis=0)
    cf = pd.DataFrame(cf,index=np.unique(section_names), columns= np.unique(hdbscan.labels_))
    # purity for each cluster
    purity = [purity_score(train_section_large[0].values[hdbscan.labels_ == k],
                           hdbscan.labels_[hdbscan.labels_ == k]) for k in np.unique(hdbscan.labels_)]
    # sort columns by purity
    cf = cf[np.unique(hdbscan.labels_)[np.argsort(purity)]]
    # sort rows by occurence in section (df not train)
    section_count = pd.DataFrame([str(s) for s in df['section_large']])
    section_count = Counter(section_count[0].values)             
    section_order = list(dict(section_count.most_common()).keys())
    cf = cf.loc[section_order]
    # plot heatmap
    plt.figure(figsize=(18,7))
    sns.heatmap(cf,
                cmap="viridis",
                annot_kws={'fontsize':11},
                square=True,
                linewidth=0.01,
                linecolor="#222")
    plt.title(f"Window: {w+window_size-1}")
    plt.show()
    
    n_labels = np.unique(hdbscan.labels_)
    n_obs = []
    purity = []
    mix = []
    # calculate the centroids for evry cluster
    centroids = np.zeros((len(n_labels[1:]),n_comp))
    for i,k in enumerate(n_labels[1:]):
        centroids[i,:]= np.mean(X_train[hdbscan.labels_==k,:],0)
        n_obs.append(sum(hdbscan.labels_==k))
        purity.append(purity_score(train_section_large.values[hdbscan.labels_==k],hdbscan.labels_[hdbscan.labels_==k]))
        mix.append(dict(Counter(train_section_large[0].values[hdbscan.labels_==k])))
    # withdraw 5 clusters with equal posision w.r.t purity 
    n=5
    terms = vectorizer.get_feature_names_out()
    ind = np.argsort(LSA.inverse_transform(centroids))[:,::-1][:,:n]
    df_eval = pd.DataFrame(terms[ind])
    df_eval['purity'] = purity
    df_eval['n_obs'] = n_obs
    df_eval['mix'] = mix
 
    c = df_eval.sort_values('purity').iloc[[i*int(len(df_eval)/5) for i in range(5)]]
    display(c.drop(["mix"],axis=1))
